In [ ]:
import chess
import chess.uci
import chess.pgn
import os
import csv
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import math
import time 


In [ ]:
#fileDir = os.path.dirname(os.path.realpath('__file__'))
pgns_all_path = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/lichess_db_standard_rated_2017-03.pgn"
#abs_file_path = os.path.join(fileDir, rel_path)
#pngs = open(abs_file_path, 'r')

pgns = open(pgns_all_path)


writing_path = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/stockfish_performances_threeseconds.txt"
stockfish_performances = open(writing_path, 'a')


In [ ]:
handler = chess.uci.InfoHandler()
engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
engine.info_handlers.append(handler)

In [ ]:
evaltime = 3000 #ms
#evaluation = engine.go(movetime=evaltime)

In [ ]:
def check_elos(game):
    print("white elo: " + game.headers["WhiteElo"])
    print("black elo: " + game.headers["BlackElo"])

In [ ]:
def get_performances(game): 
    handler = chess.uci.InfoHandler()
    engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
    engine.info_handlers.append(handler)
    
    board = game.board()

    score = 0
    prev_score = 0
    
    wp = []
    bp = []
    evals = []
    wp.append(game.headers["WhiteElo"])
    bp.append(game.headers["BlackElo"])
    
    wp.append(game.headers["ECO"])
    bp.append(game.headers["ECO"])

    evaluation = engine.go(movetime=evaltime)
    time.sleep(evaltime/1000)
    score = handler.info["score"][1].cp
    evals.append(score)

    move_count = 0
    prev_score = score
    
#     print("Beginning game with score: " + str(prev_score))
    
    for move in game.main_line():
        board.push(move)
        engine.position(board)
        move_count += 1
        evaluation = engine.go(movetime=evaltime)
        time.sleep(evaltime/1000)

        score = handler.info["score"][1].cp
        if score is None:
#             print("Finishing game...")
            break
        if move_count % 2 == 1:
            score = -score
        evals.append(score)
        dif = score - prev_score
        # print("dif calculated as: " + str(score) + " minus " + str(prev_score))
        # performance.append("%.2f" % dif)
        if move_count % 2 == 1:
            wp.append("%.2f" % dif)
        elif move_count % 2 == 0:
            bp.append("%.2f" % -dif)
        prev_score = score

#     print(wp)
#     print(bp)
    return [wp, bp]

In [ ]:
def skip_to_game(pgn_file, game_num):

    offsets = chess.pgn.scan_offsets(pgn_file)
    for i in range(game_num):
        book_mark = next(offsets)
        if i == (game_num - 1):
            pgn_file.seek(book_mark)
    #return pgn_file
# first_game_offset = next(offsets)
# second_game_offset = next(offsets)






In [ ]:
pgns = open(pgns_all_path)
skip_to_game(pgns, 604)



for i in range(1):
    game = chess.pgn.read_game(pgns)
    check_elos(game)
    

    

In [ ]:
starting_position = 720

for i in range(200):
    
    #pgns = open(pgns_all_path)

    with open(pgns_all_path) as pgns:
        skip_to_game(pgns, starting_position + i)

        game = chess.pgn.read_game(pgns)
        performances = get_performances(game)
    
        for p in performances:
            stockfish_performances.write(str(p) + "\n")
        stockfish_performances.flush()
    
        print("Finished game: " + str(i + starting_position))


In [ ]:
import psutil



In [ ]:
for proc in psutil.process_iter():
    print proc.open_files()

In [ ]:


elo_and_performance = []
for i in xrange(1):
    pw = []
    pb = []
    current_game = chess.pgn.read_game(pgns)
    try:
        pw.append(float(current_game.headers["WhiteElo"]))
    except:
        pw.append(0.0)
    try:
        pb.append(float(current_game.headers["BlackElo"]))
    except:
        pb.append(0.0)
    try:
        pw.append(float(evals[i][0]))
    except:
        pw.append(0.0) 
    for j in xrange (len(evals[i])):
        if j == 0:
            continue
        try:
            move_value = float(evals[i][j]) - float(evals[i][j-1])
        except:
            move_value = 0.0
        if j % 2 != 0: 
            pb.append(-move_value)
        else:
            pw.append(move_value)
    elo_and_performance.append(pw)
    elo_and_performance.append(pb)

elo_and_performance = array(elo_and_performance)

print(elo_and_performance)


In [ ]:


print 'best move: ', board.san(evaluation[0])
print 'evaluation value: ', handler.info["score"][1].cp/100.0
print 'Corresponding line: ', board.variation_san(handler.info["pv"][1])